In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from astropy.table import Table, vstack
from astroquery.vizier import Vizier
from astropy import units as u

import swasputils

In [3]:
CATALOGUES = [
    'J/BaltA/9/646/catalog',
    'J/ApJS/247/44/table2',
]

In [4]:
source_table = Table.read(os.path.join(swasputils.CACHE_LOCATION, 'source_coords.fits'))
source_table = source_table[:10000]

In [5]:
results = []
SPLIT_SIZE = 1000
for i in range(int(len(source_table) / SPLIT_SIZE) + 1):
    sources = source_table[i * SPLIT_SIZE : (i+1) * SPLIT_SIZE]
    results.append(Vizier.query_region(sources, radius=1*u.arcsec, catalog=CATALOGUES))

In [16]:
stacked_results = {}
for catalogue in CATALOGUES:
    stacked_results[catalogue] = []
    for result in results:
        try:
            stacked_results[catalogue].append(result[catalogue])
        except TypeError:
            pass
    stacked_results[catalogue] = vstack(stacked_results[catalogue])
stacked_results

{'J/BaltA/9/646/catalog': <Table length=51>
   _q     GCVS    magMax l_magMin  magMin ...  DEJ2000    RV    HIP   Simbad
                   mag              mag   ...    deg    km / s              
 int32    str9   float32   str1   float32 ...  float64  int16  int32   str6 
 ----- --------- ------- -------- ------- ... --------- ------ ------ ------
    81 IW    Cas   11.40        <    14.7 ...  48.64637    -19     -- Simbad
     1 R     And    5.80             14.9 ...  38.57712     -8   1901 Simbad
     1 T     And    7.70             14.5 ...  26.99605    -90     -- Simbad
     1 U     And    8.90             15.0 ...  40.71956     -6     -- Simbad
     1 V     And    9.00             15.2 ...  35.65333     18     -- Simbad
     1 W     And    6.70             14.6 ...  44.30494    -29  10687 Simbad
     1 X     And    8.30             15.2 ...  47.01267     -5     -- Simbad
     1 Y     And    8.20             15.1 ...  39.34409     -7     -- Simbad
     1 RR    And    8.40        